# Experiment analysis: Size Varying sprite discs
## animals are tectum MTZ ablated SAGFF(LF)81C with UAS:NTR-mcherry and controls (no mcherry expression visible)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import glob
from datetime import datetime

#
# load custom modules for social behavior analysis
# enter path to your local repository here
os.chdir(r'C:\Users\johannes\Dropbox\python\zFishBehavior\dishGroupBehavior')
# ----------------------------


import models.experiment as xp
import models.experiment_set as es
import functions.matrixUtilities_joh as mu
import functions.paperFigureProps as pfp

#
# notebook configuration
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

#
# custom paper style plotting
pfp.paper()

Pre-Analyze all experiments only if necessary, this takes a couple of minutes! Experiment summary csv files are saved to disk.

In [ ]:


# this example analysis has been tested on a subset of the raw data from:
    # Larsch&Baier 2018
    # Biological_motion_as_an_innate_perceptual_mechanism_driving_social_affiliation
    # available at figshare: https://doi.org/10.6084/m9.figshare.6939923.v1
    
# 20170717_15Animal_skype_age
# This experiment is part of figure 1E-G of the paper
# In this experiment, animals of ages 17 and 20 dpf were tested in pair-wise skype interactions for mutual attraction.
    
# ENTER DATA PATH HERE

startDir = os.path.normpath(r'E:\AddYourPathHere\20170717_15Animal_skype_age\\')
# -------------------------------------


# collect meta information and save to csv file
info = pd.DataFrame()

info['epiDur'] = [5]      # duration of individual episodes (default: 5)
info['recomputeAnimalSize'] = 1 # want to re-compute animal size from video? default: 1
info['arenaDiameter_mm'] = 100 # arena diameter in mm, default: 100
info['SaveNeighborhoodMaps'] = 1 # compute and save neighbor density plots, default: 1
info['ComputeBouts'] = 1 # compute bout frequency, default: 1

birthDay = np.array(['2017-6-27', '2017-6-30'])   # birth date of animals. List all dates used in experiment, then assign below.
birthTime = '09-00'         # birth time of animals. Use 9 am per default.

#
# specify birth day for each animal using indices from birth date list.
birthIndex = np.array([ 0, 0, 0, 0, 0,
                        0, 1, 1, 1, 1,
                        1, 1, 1, 1, 1])

birthDay_all = [birthDay[x]+'-'+birthTime for x in birthIndex]
birthDay_all=' '.join(birthDay_all)

# treatment of animals. List all treatments used in experiment, then assign below
treatName = np.array(['a', 'b', 'c', 'd'])

#
# specify experimental treatment using indices from treatment names
treatment = np.array([0, 1, 2, 3, 0,
                      1, 2, 3, 0, 1,
                      2, 3, 0, 1, 2])

In [ ]:
aviPath = glob.glob(startDir+'\\*.avi')  # infer avi file - should be the only .avi file 
if len(aviPath) == 0:
    aviPath=['']
    print('no avi file found. cannot recompute animal size.')
    info['recomputeAnimalSize'] = 0
    
    
posPath = glob.glob(startDir+'\\PositionTxt*.txt')  # infer PositionTxt tile name
PLPath = glob.glob(startDir+'\\PL*.txt')  # infer pair list file name

info['aviPath'] = aviPath   #avi
info['txtPath'] = posPath
info['pairList'] = PLPath
info['birthDayAll'] = birthDay_all
#parse experiment time from file name
head,tail=os.path.split(posPath[0])
tmp=tail[tail.find('ROI')+3:-4]
time = datetime.strptime(tmp, '%Y-%m-%dT%H_%M_%S')
time= datetime.strftime(time, '%Y-%m-%d %H:%M:%S')
info['expTime']=time

csvFile = os.path.join(startDir,'csvProcess_fileList.csv')
info.to_csv(csvFile, encoding='utf-8')

print('Metadata saved to:', csvFile)

info

In [ ]:
expSet = es.experiment_set(csvFile=csvFile)
print('done reading experiment')

In [ ]:
expSet.experiments[0].pair[0].IAD_m()

In [ ]:
summaryFile = glob.glob(posPath[0][:-4]+'_siSummary_epi*')[0]
print(summaryFile)
df = pd.read_csv(summaryFile,index_col=0,sep=',')
df['episode']=[x.strip().replace('_','') for x in df['episode']]
df.head()

In [ ]:
# add treatment column using animalIndex column and manually entered info from above

df['treatment']=treatName[treatment[df.animalIndex]]
df.head(6)

In [ ]:
d=df.time
r=datetime(int(df.time[0][:4]),1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]
t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df.head()

## Habituation or Fatigue within 20 hours?

Plot shoaling index during closed loop skype episodes over time.

In [ ]:
sns.tsplot(data=df, time="t3",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,24])
plt.axhline(0,ls=':',color='gray')

In [ ]:
#Limit analysis to a time window (typically ignore fist 45 minutes and times later than 350 minutes)
tStart=45
tEnd=350
idx=(df['inDishTime']<tEnd) & (df['inDishTime']>tStart) & (df['animalIndex']!=11)
dfDR=df[idx]

# Plot individual 5 minute segments for all animals
## here, using 'age' to sub-divide the data in each category

In [ ]:
sns.swarmplot(data=dfDR,
              x='episode',
              y='si',
              hue='age',
              dodge=1)
plt.axhline(0,ls=':')

# Use manually assigned 'treatment' groups to sub divide the data
### (for illustration only, this doesn't make sense in the demo data set)

In [ ]:
sns.swarmplot(data=dfDR,
              x='episode',
              y='si',
              hue='treatment',
              dodge=1)
plt.axhline(0,ls=':')

# average episodes over animals first
### generally, using n = number of animals whenever possible 

In [ ]:
dfAnimalAverage=dfDR.groupby(['episode','animalIndex','treatment'],sort=True).mean().reset_index()
sns.swarmplot(data=dfAnimalAverage,
              x='episode',
              y='si',
              hue='age',
              dodge=1)
plt.axhline(0,ls=':')

In [ ]:
sns.pointplot(data=dfAnimalAverage,
              x='episode',
              y='si',
              hue='age')
plt.axhline(0,ls=':')


In [ ]:
#individual animals
dfDR.groupby(['episode','animalIndex'],sort=True)['si'].mean().unstack().plot()

In [ ]:
sns.swarmplot(data=dfAnimalAverage,
              x='episode',
              y='si',
              hue='treatment',
              dodge=1)

sns.pointplot(data=dfAnimalAverage,
              x='episode',
              y='si',
              hue='treatment')

plt.axhline(0,ls=':')

# compare speed between groups

In [ ]:
sns.pointplot(data=dfAnimalAverage,
              x='episode',
              y='avgSpeed',
              hue='treatment')
plt.axhline(0,ls=':')

#plt.ylabel('average Speed [mm/sec]')

# thigmotaxis index

In [ ]:
sns.pointplot(data=dfAnimalAverage,
              x='episode',
              y='thigmoIndex',
              hue='treatment')
plt.axhline(0,ls=':')

# bout duration

In [ ]:
sns.pointplot(data=dfAnimalAverage,
              x='episode',
              y='boutDur',
              hue='age',
              estimator=np.median, # bout duration can be heavily influenced by outliers
             ci=None)
plt.axhline(0,ls=':')

# correlate size with attraction

In [ ]:
# this only works if size was calculated from video

sns.pairplot(dfAnimalAverage,vars=["anSize", "si"])

# Plot average neighborhood maps for all animals

In [ ]:
# get all the maps from the expSet data structure
# (this data is also stored in a .npy file)
nmatAll=np.array([y.animals[0].ts.neighborMat() for y in expSet.experiments[0].pair])

In [ ]:
levels=df['episode'].unique()
ans=df['animalIndex'].unique()
avg=np.zeros((len(ans),len(levels),nmatAll.shape[1],nmatAll.shape[2]))


In [ ]:
for an in ans:
    for i in range(len(levels)):
        ix=np.where((df['episode']==levels[i]) & (df['animalIndex']==an) & idx)[0]
        avg[an,i,:,:]=nmatAll[ix,:,:].mean(axis=0)


In [ ]:
fig, axes = plt.subplots(nrows=15, ncols=7, sharex='col', sharey=True,figsize=(10, 30))
m=np.nanpercentile(avg,95)
trLab=treatName
for an in ans:
    for i in range(len(levels)):
        axes[an,i].imshow(avg[an,i,:,:],clim=[0,m],extent=[-31,31,-31,31])
        axes[an,i].set_title('a:'+str(an)+trLab[treatment[an]]+ 's:'+levels[i][-2:],fontsize=10)

In [ ]:
levels=df['episode'].unique()
treat=treatNames
avgT=np.zeros((len(treat),len(levels),nmatAll.shape[1],nmatAll.shape[2]))


In [ ]:
df['treatment']

In [ ]:
for an in range(len(treat)):
    for i in range(len(levels)):
        ix=np.where((df['episode']==levels[i]) & (df['treatment']==treat[an]) & idx)[0]
        avgT[an,i,:,:]=nmatAll[ix,:,:].mean(axis=0)
        

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower ofset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax/(vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highets point in the colormap's range.
          Defaults to 1.0 (no upper ofset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
import matplotlib.gridspec as gridspec

pfp.paper()
inToCm=2.54

ncols=len(df.episode.unique())
nrows=len(df.treatment.unique())

outer = gridspec.GridSpec(2, 2, width_ratios = [5,.1], wspace = 0.05) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(nrows, ncols, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])

fig = plt.figure(figsize=(11/inToCm,11/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(ncols*nrows)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.bwr
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))

trLab=treatName
pal=['gray','r','g','m']
for an in range(len(treat)):
    for i in range(len(levels)):
        ind=i+(ncols*an)
        im = axes[ind].imshow(avgT[an,i,:,:],clim=[0,m],extent=[-31,31,-31,31],origin='lower')#,cmap=cmap)
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')

        if i==0:
            axes[ind].set_title(trLab[an],fontsize=8,color=pal[an])
            
        if (i==5)&(an==0):
            axes[ind].set_title('neighbor density',fontsize=9)

cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.1]))

plt.subplots_adjust(wspace=0, hspace=0.1)

social=df[idx].groupby(['treatment','episode','animalIndex']).si.mean().reset_index()
social['xpretty']=[int(ss[-2:])/2. for ss in social.episode]
sns.swarmplot(data=social,
              x='xpretty',
              hue='treatment',
              y='si',
              zorder=1,
              linewidth=1,
              edgecolor='gray',
              ax=axesSP,
              palette=pal,
              alpha=0.7)

sns.pointplot(x="xpretty", y="si", hue='treatment',data=social,ci=None,zorder=100,scale=2,ax=axesSP,palette=pal,
              linewidth=1,edgecolor='gray')
axesSP.spines['top'].set_color('white')
axesSP.spines['bottom'].set_color('white')
axesSP.spines['right'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)

axesSP.yaxis.tick_left()
axesSP.set_xlabel('dot diameter [mm]')
axesSP.set_ylabel('attraction')
handles, labels = axesSP.get_legend_handles_labels()
axesSP.legend(handles[:4], labels[:4])

axesSP.axhline(0,ls=':',color='k')


In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2/inToCm,4.5/inToCm))

social=df[idx].groupby(['treatment','animalIndex']).avgSpeed.mean().reset_index()
sns.boxplot(y="avgSpeed", x='treatment',data=social,ax=ax,palette=pal,linewidth=2)


# Select which box you want to change    
for i,artist in enumerate(ax.artists):
# Change the appearance of that box
    artist.set_edgecolor('k')
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        line.set_color('k')
        line.set_mfc('k')
        line.set_mec('k')

sns.swarmplot(data=social,x='treatment',y='avgSpeed',zorder=100,linewidth=1,ax=ax,palette=pal,alpha=0.7,edgecolor='k')

plt.xlabel('')
plt.xticks([])
plt.ylabel('average Speed \n [mm/sec]')

plt.ylim([0,7])
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)


In [ ]:
fig, axes = plt.subplots(nrows=treatNum, ncols=7, sharex=True, sharey=True,figsize=(10,10))
m=np.nanpercentile(avg,95)
trLab=treatNames
for an in range(len(treat)):
    for i in range(len(levels)):
        profile=avgT[an,i,:,29:31].mean(axis=1)
        axes[an,i].plot(profile,np.arange(profile.shape[0])-30)
        axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        axes[an,i].axhline(0,ls=':',color='gray')


In [ ]:
fig, axes = plt.subplots(nrows=treatNum, ncols=len(levels), sharex='col', sharey=True,figsize=(10, 10))
m=np.nanpercentile(avg,95)
trLab=treatNames
for an in range(len(treat)):
    for i in range(len(levels)):
        axes[an,i].plot(avgT[an,i,29:31,:].mean(axis=0))
        axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        axes[an,i].axvline(30)